In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchaudio torchvision roboflow ultralytics

In [ ]:
# Dataset import (https://universe.roboflow.com/mohamed-traore-2ekkp/taco-trash-annotations-in-context/dataset/15)

from roboflow import Roboflow
rf = Roboflow(api_key="Enter your key") # Enter your roboflow api key here
project = rf.workspace("mohamed-traore-2ekkp").project("taco-trash-annotations-in-context")
version = project.version(15)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to TACO:-Trash-Annotations-in-Context-Dataset-15 in yolov11:: 100%|██████████| 7206/7206 [00:01<00:00, 4606.91it/s]


In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
#Training the model

model = YOLO("/content/drive/MyDrive/best_heavy_59classes.pt") # load pretrained model (YOLO11n-seg)

# Train the model (single gpu)
results = model.train(data="/content/TACO:-Trash-Annotations-in-Context-Dataset-15/data.yaml", epochs=50, imgsz=640)

### Visualization ###

In [ ]:
# Waste Bin Mapping (German Mülltrennung System)
bin_map = {
    "Yellow Bin (Gelbe Tonne)": [
        "Aluminium blister pack", "Aluminium foil", "Carded blister pack", "Clear plastic bottle",
        "Disposable food container", "Disposable plastic cup", "Drink Carton", "Drink can",
        "Foam cup", "Meal carton", "Metal lid", "Metal bottle cap", "Other plastic bottle",
        "Other plastic container", "Other plastic cup", "Other plastic wrapper", "Plastic bottle cap",
        "Plastic film", "Plastic glooves", "Plastic lid", "Plastic straw", "Polypropylene bag",
        "Pop tab", "Single-use carrier bag", "Six pack rings", "Spread tub", "Squeezable tube",
        "Tupperware"
    ],
    "Grey Bin (Restmüll)": [
        "Cigarette", "Garbage bag", "Shoe", "Unlabeled litter", "Plastified paper bag",
        "Styrofoam piece", "Rope & strings", "Foam food container", "Other plastic",
        "Pizza box", "Tissues"
    ],
    "Green Bin (Biotonne)": [
        "Food waste"
    ],
    "Blue Bin (Papiertonne)": [
        "Egg carton", "Normal paper", "Other Carton", "Paper Bag", "Paper cup", "Paper straw",
        "Pizza box", "Toilet tube", "Magazine paper", "Wrapping paper", "Corrugated carton"
    ],
    "Glascontainer": [
        "Glass bottle", "Glass cup", "Glass jar", "Broken glass"
    ],
    "Hazardous Waste (Sondermüll)": [
        "Battery", "Aerosol", "Scrap metal"
    ],
    "Deposit Return (Pfand)": [
        "Drink can", "Clear plastic bottle", "Glass bottle", "Food Can"
    ]
}

# Bin colors in BGR format (for OpenCV)
bin_colors = {
    "Yellow Bin (Gelbe Tonne)": (0, 255, 255),       # Yellow
    "Grey Bin (Restmüll)": (128, 128, 128),          # Gray
    "Green Bin (Biotonne)": (0, 255, 0),             # Green
    "Blue Bin (Papiertonne)": (200, 0, 0),           # Blue (bright)
    "Glascontainer": (0, 0, 0),                      # Black
    "Hazardous Waste (Sondermüll)": (0, 0, 255),     # Red
    "Deposit Return (Pfand)": (255, 0, 255)          # Purple/Magenta
}

class_to_bin = {cls: bin_type for bin_type, cls_list in bin_map.items() for cls in cls_list} #class name gives bin type

from ultralytics import YOLO
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load model
model = YOLO("/content/best_heavy_59classes.pt") # enter path to your model

# Load image
image_path = "/content/IMG-20221229-WA0003_jpg.rf.fce2a8664e953ba379b83eb2e29a32cb.jpg" # enter image path for testing
image = cv2.imread(image_path)
image = cv2.resize(image, (640, 640))

# Run prediction
results = model(image_path)
result = results[0]

# Get class names
class_names = model.names

# Inverse mapping: class name → bin
class_to_bin = {cls: bin_name for bin_name, class_list in bin_map.items() for cls in class_list}

# Get results
masks = result.masks.data.cpu().numpy()     # [N, H, W]
boxes = result.boxes.xyxy.cpu().numpy()     # [N, 4]
scores = result.boxes.conf.cpu().numpy()    # [N]
class_ids = result.boxes.cls.cpu().numpy()  # [N]

overlay = image.copy()

for i in range(len(class_ids)):
    class_id = int(class_ids[i])
    score = scores[i]
    mask = masks[i]
    box = boxes[i].astype(int)

    class_name = class_names[class_id]

    bin_type = next((bin_name for bin_name, items in bin_map.items() if class_name in items), "Cannot detect trash type")
    color = bin_colors.get(bin_type, (255, 255, 255))  # fallback to white

    # Apply colored mask (blended)
    mask_3c = np.stack([mask] * 3, axis=-1)  # [H, W, 3]
    color_array = np.array(color, dtype=np.uint8).reshape(1, 1, 3)
    colored_mask = (mask_3c * color_array).astype(np.uint8)

    overlay = cv2.addWeighted(overlay, 1.0, colored_mask, 0.5, 0)

    # Draw bounding box
    cv2.rectangle(overlay, (box[0], box[1]), (box[2], box[3]), color, 2)

    # Draw label (class name + score + bin)
    label = f"{class_name} ({int(score * 100)}%)\n{bin_type}"
    for j, line in enumerate(label.split("\n")):
        text_pos = (box[0], box[1] - 10 - 20 * j)
        cv2.putText(overlay, line, text_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

# Show the image
plt.figure(figsize=(14, 10))
plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("YOLOv11 Mülltrennung Classification Overlay")
plt.show()